# Convert a Constellation graph for Webstellation
Use the Constellation REST API to get some graph, vertex, and transaction attributes so we can use Webstellation to visualise a graph in a web browser.

In [ ]:
import json
from PIL import Image
import io
import re
import pandas as pd
import numpy as np

import constellation_client

In [ ]:
# Constellation icons are this big.
#
ICON_SIZE = 256

# We build a texture map this big.
# Even low-end GPUs should be able to cope with this.
#
TEXTURE_SIZE = 4096

TEXTURE_ICONS = TEXTURE_SIZE // ICON_SIZE

print(f'Maximum number of icons: {TEXTURE_ICONS**2}')

SAVE_DIR = '..'

In [ ]:
cc = constellation_client.Constellation()

In [ ]:
# named_colors = cc.call_service('list_named_colors').json()

# for name, html in named_colors.items():
#     gl = htmlToGl_4(html)
#     print(name, html, gl)

In [ ]:
def colorToFloat4(c):
    """Convert a color name or an HTML #rrggbb color or a str(list of floats) to a GL list of floats between 0.0 and 1.0 color, and alpha."""
    
    def get_color_names():
        if not hasattr(colorToFloat4, 'named_colors'):
            colorToFloat4.named_colors = cc.call_service('list_named_colors').json()
        
        return colorToFloat4.named_colors
        
    if not c:
        raise ValueError(f'Bad color "{c}"')
    
    if c[0]=='[' and c[-1]==']':
        color = c[1:-1].split(',')
        color = [float(i) for i in color]
        if len(color)==3:
            color.append(1.0)
        
        return color
        
    if len(c)!=7 or not c.startswith('#'):
        cn = get_color_names()
        if c not in cn:
            raise ValueError(f'Bad HTML color "{c}"')
        
        c = cn[c]
    
    r = int(c[1:3], 16)
    g = int(c[3:5], 16)
    b = int(c[5:7], 16)
    
    return [r/255, g/255, b/255, 1.0]

def float4ToHtml(c):
    """Convert a list of 4 floats color to an HTML #rrggbb color."""
    
    return '#' + ''.join(f'{int(i*255):02x}' for i in c[:3])

def parse_blaze(b):
    """Extract the color and angle from the blaze string."""
    
    if not b:
        return None

    angle, color = b.split(';')
    angle = float(angle)
    color = colorToFloat4(color)
    
    return {'angle':angle, 'color':color}

## Graph
Collect graph attributes that affect the visualisation.

In [ ]:
gdf = cc.get_graph_attributes()
gdf.to_dict(orient='records')

In [ ]:
# The camera details.
#
def get_camera(s):
    """Get the camera eye,centre,up from the camera string."""
    
    pos = re.compile(r'(\w+): 3f\[(.+?)\]')
    camera = {}
    for name,loc in pos.findall(s):
        camera[name] = [float(f) for f in loc.split(',')]

    return camera

camera = get_camera(gdf.loc[0, 'camera'])

# The background color.
#
background_color = gdf.loc[0, 'background_color']

# The attribute that specifies the node color (if notspecified, 'color').
#
node_color_attr = gdf.loc[0, 'node_color_reference']
if node_color_attr is None:
    node_color_attr = 'color'

# The attribute that specifies the link color.
#
link_color_attr = gdf.loc[0, 'transaction_color_reference']
if link_color_attr is None:
    link_color_attr = 'color'

# The attribute that specifies the node label.
# We only use the first attribute of the bottom labels, and we ignore the size.
#
labels = gdf.loc[0, 'node_labels_bottom']
if '|' in labels:
    labels = labels.split('|')[0]

labels = labels.split(';')
label_attr = labels[0]
label_color = float4ToHtml(colorToFloat4(labels[1])) # Because BABYLON GUI is HTML.

print(f'Camera: {camera}')
print(f'Background: {background_color}')
print(f'Node color attr: "{node_color_attr}"')
print(f'Link color attr: "{link_color_attr}"')
print(f'Label attr: "{label_attr}" {label_color}')

## Vertices
Some attribute names (eg icon, x) are fixed; we need these specific attributes.

Some attribute names (eg the attributes being used for the label and node color) vary, and may even use a fixed attribute (like using the icon for the label).

As well as including the fixed attributes, we need to get the varying attributes and give them fixed names to make it easy for the front-end.

In [ ]:
# Get the graph + vertex + transaction attributes.
#
all_attrs = cc.get_attributes()
all_attrs

In [ ]:
fixed_vx_attrs = ['[id]', 'background_icon', 'blaze', 'icon', 'nradius', 'x', 'y', 'z']

# Add the attributes that start with an upper-case letter.
# These are probably the ones that people want to see.
#
extra_attrs = [a for a in all_attrs if a.startswith('source')]
extra_attrs = [a.replace('source.', '') if a.startswith('source.') else a for a in extra_attrs]
extra_attrs = [a for a in extra_attrs if 'A'<=a[0]<='Z']
fixed_vx_attrs.extend(extra_attrs)

vx_attrs = [f'source.{i}' for i in fixed_vx_attrs]

if label_attr not in fixed_vx_attrs:
    vx_attrs.append(f'source.{label_attr}')

if node_color_attr not in fixed_vx_attrs:
    vx_attrs.append(f'source.{node_color_attr}')

vxdf = cc.get_dataframe(vx=True, attrs=vx_attrs)
vxdf = vxdf.rename(columns=lambda name:name.replace('source.', '', 1) if name.startswith('source.') else name)

vxdf.blaze = vxdf.blaze.apply(parse_blaze)

# Datetime NaT values don't serialise into JSON, so turn them into None.
#
dtcols = vxdf.select_dtypes(include=[np.datetime64, 'datetime', 'datetime64', 'datetimetz', 'datetime64']).columns
for col in dtcols:
    vxdf[col] = vxdf[col].astype('str').replace('NaT', '')

graph_name = f'graph-{len(vxdf)}'

# print([col for col in vxdf.dtypes.index if 'A'<=col[0]<='Z'])
vxdf.head()

In [ ]:
vx_noted = list(vxdf.head(3)['[id]'])
vx_noted

## Icons

In [ ]:
def make_texture_atlas(icons, atlas_name):
    """Given a collection of icons of size ICON_SIZE, create a texture atlas.
    
    TODO put the background icons in first?
    
    :returns: the width,height of the atlas, a map from name to cell_index
    """
    
    icons = set(icons)
    
    N = len(icons)
    W = min(N, TEXTURE_ICONS)
    H = N % W + 1
    if H >= TEXTURE_ICONS:
        raise ValueError('Too many icons')
    
    print(N, W, H)
    
    sheet = Image.new('RGBA', (W*ICON_SIZE, H*ICON_SIZE))
    print(f'Created texture atlas {sheet.size}')
    
    icon_map = {}
    for i, icon_name in enumerate(icons):
        x = i % W
        y = i // W
        cell_index = y*W + x
        print('.', x, y, cell_index, icon_name)
        
        img = cc.get_icon(icon_name)
        with io.BytesIO(img) as buf:
            img = Image.open(buf)
            
            ISIZE = ICON_SIZE, ICON_SIZE
            if img.size>ISIZE:
                img = img.thumbnail(ISIZE)
            
            if img.size[0]<ICON_SIZE or img.size[1]<ICON_SIZE:
                img_size = Image.new('RGBA', ISIZE, (255, 255, 255, 0))
                dx = (ICON_SIZE-img.size[0])//2
                dy = (ICON_SIZE-img.size[1])//2
                img_size.paste(img, (dx, dy))
                img = img_size
        
            print('  ', (x*ICON_SIZE, y*ICON_SIZE))
            sheet.paste(img, (x*ICON_SIZE, y*ICON_SIZE))
            icon_map[icon_name] = y*N + i
    
    print('Save texture atlas to {atlas_name}')
    sheet.save(atlas_name)
    
    return W, H, icon_map

In [ ]:
icons = set(vxdf['icon']) | set(vxdf['background_icon'])
sprite_atlas = f'{SAVE_DIR}/{graph_name}-atlas.png'
width, height, icon_map = make_texture_atlas(icons, sprite_atlas)
vxdf['fg_icon_index'] = vxdf.icon.apply(lambda icon_name:icon_map[icon_name] if icon_name else 0)
vxdf['bg_icon_index'] = vxdf.background_icon.apply(lambda icon_name:icon_map[icon_name] if icon_name else 0)

In [ ]:
vxs = {}
for vx in vxdf.to_dict(orient='records'):
    vx_id = vx.pop('[id]')
#     print(vx)
    vxs[vx_id] = vx
# vxs

## Transactions

In [ ]:
mix_color = gdf.loc[0, 'mix_color']

tx_attrs = ['source.[id]', 'destination.[id]', 'transaction.directed']

user_tx_attrs = [a for a in all_attrs if a.startswith('transaction.')]
user_tx_attrs = [a.replace('transaction.', '') if a.startswith('transaction.') else a for a in user_tx_attrs]
user_tx_attrs = [a for a in user_tx_attrs if 'A'<=a[0]<='Z']
print(user_tx_attrs)

extra_attrs = [f'transaction.{i}' for i in user_tx_attrs]
tx_attrs.extend(extra_attrs)

tx_attrs.append(f'transaction.{link_color_attr}')

txdf = cc.get_dataframe(tx=True, attrs=tx_attrs)
txdf = txdf.rename(columns=lambda name:name.replace('transaction.', '', 1) if name.startswith('transaction.') else name)
txdf = txdf.rename(columns={'source.[id]':'sid_', 'destination.[id]':'did_'})

# Datetime NaT values don't serialise into JSON, so turn them into None.
#
dtcols = txdf.select_dtypes(include=[np.datetime64, 'datetime', 'datetime64', 'datetimetz', 'datetime64']).columns
for col in dtcols:
    txdf[col] = txdf[col].astype('str').replace('NaT', '')


print(txdf.shape)
txdf.head(10)

In [ ]:
def line_directions(df, lines):
    """What directions are the transactions in the given rows?

    Return a string containing one or both of '>' (directed) or '|' (undirected).
    """

    directed = set(df.loc[lines, 'directed'])
    is_dir = '>' if True in directed else ''
    is_not_dir = '|' if False in directed else ''
    
    return is_dir + is_not_dir

def line_color(df, lines, color_attr):
    """Get the (possibly mixed) color of the given rows."""

    colors = list(df.loc[lines, color_attr])
    n_colors = len(set(tuple(c) for c in colors))
    if n_colors==1:
        return colors[0]
    else:
        return mix_color

def get_lines(txdf, color_attr, user_tx_attrs):
    """Get the direction(s), color, and data of the combined transactions from src to dst.
    
    If there are transactions from dst to src, get the direction(s) and color of those,
    and combine them with the src->dst direction(s) and color, because the front-end
    only draws one line.
    
    The data attributes for each transaction are written as a list of values
    in user_tx_attrs order.
    """
    
    grp = txdf.groupby(['sid_', 'did_'])
    items = dict(grp.indices.items())
    txs = []
    while items:
        rows = []
        (src,dst), tx_ixs = items.popitem()
        count = len(tx_ixs)
        
        for ix in tx_ixs:
            rows.append(list(txdf.loc[ix, user_tx_attrs]))
            
        dirs = line_directions(txdf, tx_ixs)
        color = line_color(txdf, tx_ixs, color_attr)
        if (dst,src) in items:
            tx_ixs_alt = items.pop((dst, src))
            count += len(tx_ixs_alt)
            
            for ix in tx_ixs_alt:
                rows.append(list(txdf.loc[ix, user_tx_attrs]))
            
            dirs_alt = line_directions(txdf, tx_ixs_alt)
            color_alt = line_color(txdf, tx_ixs_alt, color_attr)

            if '>' in dirs_alt:
                dirs += '<'
            if '|' in dirs_alt and '|' not in dirs:
                lines += '|'

            if color_alt!=color:
                color = mix_color
        
        txs.append({
            'sid_': src,
            'did_': dst,
            'color': color,
            'directions': dirs,
            'count': count,
            'data': rows
        })
    
    return txs

txs = get_lines(txdf, link_color_attr, user_tx_attrs)
txs

In [ ]:
# Write the JSON document for the Constellation web viewer.
#

# The user should be able to select the columns that are shown in the UI.
# For now, we'll just pick the upper-case columns.
#
user_vx_attrs = [col for col in vxdf.dtypes.index if 'A'<=col[0]<='Z']

with open(f'{SAVE_DIR}/{graph_name}.json', 'w') as f:
    json.dump({
        'vertex': vxs,
        'transaction': txs,
        'sprite_atlas': {
            'name': sprite_atlas,
            'width': width,
            'height': height
            },
        'icon': icon_map,
        'vx_noted': vx_noted,
        'camera': camera,
        'label_attr': label_attr,
        'label_color': label_color,
        'node_color_attr': node_color_attr,
        'background_color': background_color,
        'ui_attrs': user_vx_attrs,
        'ui_tx_attrs': user_tx_attrs
        }, f, indent=2)

# An example graph

In [ ]:
import networkx as nx
import pandas as pd

In [ ]:
g = nx.les_miserables_graph()
people = [[p1, p2] for p1,p2 in g.edges]
df = pd.DataFrame.from_records(people)
df.columns = ['source.Identifier', 'destination.Identifier']
df['source.Type'] = 'Person'
df['destination.Type'] = 'Person'
df['transaction.Type'] = 'Relationship'
df.head()

In [ ]:
cc = constellation_client.Constellation()
cc.put_dataframe(df)